Trying out ARMED models including Mixed Effect model from reference notebook: 
ARMED-MixedEffectsDL / ad_conversion / "model_comparison_probe_features.ipynb".

Here the Breast Cancer diagnosis binary classification problem is considerd. The aim is get MixedEffectsMLP functioning, which is currently faces some error.

In [74]:
import os
# Suppress unneeded tensorflow log info
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from armed.tfutils import set_gpu
from armed.models.mlp_classifiers import BaseMLP, ClusterCovariateMLP, DomainAdversarialMLP, MixedEffectsMLP
from armed.metrics import classification_metrics

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from scipy import stats
tf.get_logger().setLevel('ERROR')

In [75]:
data = pd.read_csv('breastcancercluster.csv', index_col = None)
data = data.drop('id',axis=1)
data = data.drop('Unnamed: 32',axis=1)
data['hospital'] = np.random.randint(1, 11, size=569)
data['diagnosis'] = data['diagnosis'].map({'M':1,'B':0})
data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,hospital
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,9
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,2
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,10
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3


In [76]:
X = data.drop('diagnosis', axis=1)
scaler = StandardScaler()
X_scaled=scaler.fit_transform(X)
y = data.diagnosis

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

# BaseMLP

In [132]:
dictCompile = {'loss':tf.keras.losses.BinaryCrossentropy(),
                'metrics': tf.keras.metrics.AUC(curve='PR', name='auprc'),
                'optimizer': tf.keras.optimizers.Adam(lr=0.001)}

In [133]:
model = BaseMLP()

In [134]:
model.compile(**dictCompile)

In [135]:
model.fit(X_train, y_train, epochs = 50, verbose=0)

In [139]:
y_pred = model.predict(X_test)
y_pred[:10] # Why not binary?!!!!!

array([[0.66697323],
       [0.9992107 ],
       [0.9767041 ],
       [0.04116708],
       [0.07228686],
       [0.99756205],
       [0.99975723],
       [0.9706528 ],
       [0.05226202],
       [0.10716418]], dtype=float32)

Here Binary Output was expected

In [101]:
# #Converting predictions into 1 and 0.

# y_preds_binary = []

# for i in y_pred:
#     if i>= 0.5:
#         y_preds.append(1)
#     else:
#         y_preds.append(0)

In [137]:
dictMetrics, youden = classification_metrics(y_test, y_pred)

In [138]:
dictMetrics

{'AUROC': 0.9876543209876543,
 'Accuracy': 0.9497354497354498,
 "Youden's index": 0.8994708994708995,
 'F1': 0.9365079365079365,
 'PPV': 0.9365079365079365,
 'NPV': 0.9629629629629629,
 'Sensitivity at Youden': 0.9365079365079365,
 'Specificity at Youden': 0.962962962962963,
 'Sensitivity at 80% Specificity': 0.984127,
 'Specificity at 80% Sensitivity': 1.0}

# Domain adversarial model

In [119]:
dictBuild = {'n_clusters': 2, 'adversary_layer_units': [8, 8, 4]}
dictCompile = {'loss_gen_weight': 10.0}
tf.random.set_seed(283)

In [120]:
model = DomainAdversarialMLP(**dictBuild)

In [121]:
model.compile(**dictCompile)

In [123]:
model.fit((X_train, y_train))

ValueError: in user code:

    c:\users\htals\.conda\envs\armedconda\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\users\htals\.conda\envs\armedconda\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\htals\.conda\envs\armedconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\htals\.conda\envs\armedconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\htals\.conda\envs\armedconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\htals\.conda\envs\armedconda\l

In [ ]:
y_pred = model.predict(X_test, y_test)

# Mixed effects model, no domain adversarial subnetwork

Error fitting and predicting the data.

In [126]:
dictBuild = {'n_features': X.shape[1],
             'n_clusters': 2,
             'adversary_layer_units': [8, 8, 4],
             'slope_posterior_init_scale': 0.1, 
             'intercept_posterior_init_scale': 0.1, 
             'slope_prior_scale': 0.15, 
             'intercept_prior_scale': 0.1,
             'kl_weight': 0.001}
dictCompile = {'loss_class_fe_weight': 1.0, 'loss_gen_weight': 0., 'loss_class_me_weight': 1.0}

In [129]:
model_class = MixedEffectsMLP
lsRandomSlopeSD = [] 
lsRandomInt = []

model = model_class(**dictBuild)
model.compile(**dictCompile)


model.fit(X_train, y_train)

OperatorNotAllowedInGraphError: in user code:

    c:\users\htals\.conda\envs\armedconda\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\users\htals\.conda\envs\armedconda\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\htals\.conda\envs\armedconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\htals\.conda\envs\armedconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\htals\.conda\envs\armedconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\htals\.con

In [ ]:
model.predict(X_test, y_test)

In [130]:
def compute_random_slope_sd(model):
    """Compute the inter-cluster variances of the ME-MLP's learned random slopes.

    Args:
        model: trained model

    Returns:
        pd.DataFrame: s.d. for each feature
    """    
    arrRandomSlopePosterior = model.randomeffects.re_slope.weights[0].numpy()
    nClusters = 2
    nFeatures = X.shape[1]
    nWeights = nClusters * nFeatures
    arrRandomSlope = arrRandomSlopePosterior[:nWeights]
    arrRandomSlope = arrRandomSlope.reshape((nClusters, nFeatures))

    return pd.DataFrame({'Feature': kfolds.x.columns, 'S.D.': arrRandomSlope.std(axis=0)})
    
def get_random_intercepts(model):
    """Get the learned random intercepts from the trained ME-MLP.

    Args:
        model: trained model

    Returns:
        pd.DataFrame: Site-specific random intercepts
    """    
    arrRandomIntPosterior = model.randomeffects.re_int.weights[0].numpy()
    nClusters = 2
    arrRandomInt = arrRandomIntPosterior[:nClusters]
    
    return pd.DataFrame({'Site': kfolds.z.columns, 'Random intercept': arrRandomInt})

In [ ]:
lsRandomSlopeSD = [compute_random_slope_sd(model)]
lsRandomInt = [get_random_intercepts(model)]